In [3]:
from keras.applications import mobilenet
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Input
from keras.models import Model, load_model
from keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
from keras.optimizers import SGD

import numpy as np
import os
import pickle
import h5py

from classes import CLASSES, LABELS_INDEX

In [2]:
image_size = (224, 224)
base_model = mobilenet.MobileNet(include_top=False, input_shape=(*image_size, 3), weights='imagenet', pooling='avg')

In [ ]:
def reorganize_data(label_file='./dataset/labels.csv',
                    base_dir='./dataset/train/',
                    train_dir=None,
                    validation_dir=None, validation_split=0.1, ext='.jpg'):
    # Read label file to get list of file name and label
    data_by_class = {}
    with open(label_file, 'r') as f:
        next(f)
        for line in f:
            filename, label = line.split(',')
            filename = filename.strip()
            label = label.strip()
            if label not in data_by_class:
                data_by_class[label] = []
            data_by_class[label].append(filename)
    # Create dirs
    validation_split = 0.1
    train_dir = train_dir or os.path.join(base_dir, 'train')
    validation_dir = validation_dir or os.path.join(base_dir, 'validation')
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    # Moving everything into its folder
    for label, files in data_by_class.items():
        label_dir_train = os.path.join(train_dir, label)
        os.mkdir(label_dir_train)
        label_dir_val = os.path.join(validation_dir, label)
        os.mkdir(label_dir_val)

        val_index = len(files) * validation_split
        for i, filename in enumerate(files):
            if ext:
                filename += ext
            old_path = os.path.join(base_dir, filename)
            if os.path.isfile(old_path):
                if (i + 1) < val_index:
                    os.rename(old_path, os.path.join(label_dir_val, filename))
                else:
                    os.rename(old_path, os.path.join(label_dir_train, filename))

In [4]:
# Freeze bottom layer
for layer in base_model.layers:
    layer.trainable = False

def get_top_model(base_model):
    top_input = Input(shape=base_model.output_shape[1:])
    x = Dense(1024, activation="relu", name='dense1024')(top_input)
    x = Dropout(0.5, name='drop05')(x)
    x = Dense(512, activation="relu", name='dense512')(x)
    x = Dropout(0.2, name='drop02')(x)
    output = Dense(len(CLASSES), activation="softmax", name='final')(x)
    model = Model(input=top_input, output=output)
    model.compile(
        loss="categorical_crossentropy",
        optimizer='adam',
        metrics=["accuracy"])
    return model

def get_full_model(base_model, top_model):
    full_output = top_model(base_model.output)
    full_model = Model(input=base_model.input, output=full_output)
    full_model.compile(
        loss='categorical_crossentropy',
        optimizer=SGD(lr=5e-5, momentum=0.9),
        metrics=["accuracy"]
    )
    return full_model


In [5]:
def predict(model, path):
    img = load_img(path, target_size=(224, 224))
    img_arr = img_to_array(img)
    inp = preprocess_input(img_arr)
    inp = np.expand_dims(inp, axis=0)
    preds = model.predict(inp)
    return preds[0]

In [8]:
train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    './dataset/images/train/', 
    target_size=image_size,
    batch_size=25,
    class_mode='categorical', shuffle=False)
val_generator = val_datagen.flow_from_directory(
    './dataset/images/validation//', 
    target_size=image_size,
    batch_size=25,
    class_mode='categorical', shuffle=False)

Found 9357 images belonging to 122 classes.
Found 1015 images belonging to 122 classes.


In [ ]:
bottleneck_features_train = base_model.predict_generator(train_generator)

In [ ]:
hf = h5py.File('bottleneck_features.h5', 'w')
hf.create_dataset('bottleneck_features_train', data=bottleneck_features_train)

In [ ]:
bottleneck_features_val = base_model.predict_generator(val_generator)
hf.create_dataset('bottleneck_features_val', data=bottleneck_features_val)
hf.close()

In [6]:
hf = h5py.File('bottleneck_features.h5', 'r')
bottleneck_features_train = hf.get('bottleneck_features_train').value
bottleneck_features_val = hf.get('bottleneck_features_val').value

In [ ]:
bottleneck_features_train == bottleneck_features_train2

In [9]:
indices = np.array(train_generator.classes)
Y_train = np.zeros((len(train_generator.classes), len(train_generator.class_indices)))
Y_train[np.arange(len(train_generator.classes)), indices] = 1
indices = np.array(val_generator.classes)
Y_val = np.zeros((len(val_generator.classes), len(train_generator.class_indices)))
Y_val[np.arange(len(val_generator.classes)), indices] = 1

In [10]:
top_model = get_top_model(base_model)

/Users/nhutrinh/dogbreeed/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fi...)`
  if sys.path[0] == '':


In [11]:
history = top_model.fit(bottleneck_features_train, Y_train, batch_size=50, epochs=40)

Epoch 1/40
9357/9357 [==============================] - 3s 298us/step - loss: 3.1524 - acc: 0.2468
Epoch 2/40
9357/9357 [==============================] - 2s 243us/step - loss: 1.5867 - acc: 0.5279
Epoch 3/40
9357/9357 [==============================] - 2s 241us/step - loss: 1.3118 - acc: 0.6057
Epoch 4/40
9357/9357 [==============================] - 2s 260us/step - loss: 1.1315 - acc: 0.6475
Epoch 5/40
9357/9357 [==============================] - 2s 242us/step - loss: 1.0197 - acc: 0.6821
Epoch 6/40
9357/9357 [==============================] - 2s 247us/step - loss: 0.9292 - acc: 0.7054
Epoch 7/40
9357/9357 [==============================] - 3s 274us/step - loss: 0.8422 - acc: 0.7344
Epoch 8/40
9357/9357 [==============================] - 3s 275us/step - loss: 0.7721 - acc: 0.7547
Epoch 9/40
9357/9357 [==============================] - 3s 268us/step - loss: 0.7237 - acc: 0.7646
Epoch 10/40
9357/9357 [==============================] - 3s 269us/step - loss: 0.6797 - acc: 0.7781
Epoch 11/

In [13]:
top_model.evaluate(bottleneck_features_val, Y_val)

1015/1015 [==============================] - 0s 225us/step


[1.1110828039094145, 0.7349753695168518]

In [14]:
top_model.save_weights('top-mblnet.w')

In [15]:
final_model = get_full_model(base_model, top_model)

/Users/nhutrinh/dogbreeed/env/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`


In [ ]:
final_model

In [16]:
label_by_index = {v:k for k,v in train_generator.class_indices.items()}
CLASSES = [label_by_index[i] for i in label_by_index]

In [ ]:
d.get_weights()

In [ ]:
final_model.save('dogbreed.h5')
# top_model = load_model('./dogbreed.h5')

In [ ]:
final_model.fit_generator(generator=train_generator, epochs=2, shuffle=True)

In [17]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(final_model, './js_model')

In [3]:
from keras.utils import CustomObjectScope
with CustomObjectScope({'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D}):
    model = load_model('dogbreed-mobilenet.h5')

In [4]:
def predict_label(model, path):
    p = predict(model, path)
    top_indices = p.argsort()[-5:][::-1]
    result = [(CLASSES[i], p[i],) for i in top_indices]
    result.sort(key=lambda x: x[1], reverse=True)
    return result


In [6]:
predict_label(model, './cuteness2.jpg')

[('malamute', 0.7079626),
 ('eskimo_dog', 0.12673402),
 ('samoyed', 0.08253971),
 ('siberian_husky', 0.07271839),
 ('collie', 0.007111568)]

In [7]:
# Have to unfreeze layer before save to js otherwise not work
for layer in model.layers:
    layer.trainable = True

In [18]:
model.save_weights('test.wh')

In [19]:
model.load_weights('test.wh')

In [9]:
model.add_weight()

<bound method Layer.add_weight of <keras.engine.training.Model object at 0x123949390>>

In [20]:
from keras.utils import CustomObjectScope
import tensorflowjs as tfjs
with CustomObjectScope({'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D}):
    tfjs.converters.save_keras_model(model, './js_model')

In [ ]:
import keras as K
import tensorflow as tf
sess = K.backend.get_session()

In [43]:
[CLASSES[i] for i in np.argsort(predict(model, './cuteness1.jpg'))[::-1][:5]]

['soft-coated_wheaten_terrier',
 'old_english_sheepdog',
 'otterhound',
 'komondor',
 'irish_wolfhound']

In [13]:
LABELS_INDEX

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8,
 'beagle': 9,
 'bedlington_terrier': 10,
 'bernese_mountain_dog': 11,
 'black-and-tan_coonhound': 12,
 'blenheim_spaniel': 13,
 'bloodhound': 14,
 'bluetick': 15,
 'border_collie': 16,
 'border_terrier': 17,
 'borzoi': 18,
 'boston_bull': 19,
 'bouvier_des_flandres': 20,
 'boxer': 21,
 'brabancon_griffon': 22,
 'briard': 23,
 'brittany_spaniel': 24,
 'bull_mastiff': 25,
 'cairn': 26,
 'cardigan': 27,
 'cat': 28,
 'chesapeake_bay_retriever': 29,
 'chihuahua': 30,
 'chow': 31,
 'clumber': 32,
 'cocker_spaniel': 33,
 'collie': 34,
 'curly-coated_retriever': 35,
 'dandie_dinmont': 36,
 'dhole': 37,
 'dingo': 38,
 'doberman': 39,
 'english_foxhound': 40,
 'english_setter': 41,
 'english_springer': 42,
 'entlebucher': 43,
 'eskimo_dog': 44,
 'flat-coated_retriever': 45,
 'french_bulldog': 46,
 'germ